In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv

In [120]:
image_list = []
label_list = []
for i in range(1,1848):
    name = 'C:/Users/sfsfk/Desktop/develope/tensorflow/ssuface/data/pencilCase/test'
    name = name + str(i) + '.jpg'
    image_list.append(name)
    label_list.append([0,1])

In [121]:
def _read_list_image(input):
    return [cv.imread(input[x], cv.IMREAD_GRAYSCALE) for x in range(1847)]

In [207]:
img = np.stack(_read_list_image(image_list))
label = np.array(label_list, dtype=np.float32)

In [212]:
img = tf.reshape(img, shape=[-1,2304])
img = tf.to_float(img,name='ToFloat')

In [243]:
print(type(img))

<class 'tensorflow.python.framework.ops.Tensor'>


In [213]:
dataset = tf.data.Dataset.from_tensor_slices((img,label))
dataset = dataset.repeat()
dataset = dataset.shuffle(2500)
dataset = dataset.batch(20)

In [214]:
iterator = dataset.make_initializable_iterator()
image_stack, label_stack = iterator.get_next()
next_element = iterator.get_next()

In [215]:
sess = tf.Session()

In [216]:
x = tf.placeholder(dtype=tf.float32, shape = [None, 2304])# 48 * 48 size image
y = tf.placeholder(dtype=tf.float32, shape = [None, 2])# yes or no

In [217]:
x_image = tf.reshape(x, [-1,48,48,1]) #벡터화된 이미지 값을 매트릭스화

In [218]:
def weight_variable(shape): #weight 정의 함수. kernel, model등에 쓰임
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

In [219]:
def bias_variable(shape): #bias 정의 함수, 위와 마찬가지로 쓰임
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [220]:
def convolution(x, kernel): #합성곱을 한칸씩 이동해서 하며 이를 통해 만들어진 feature map의 크기가 같음
    return tf.nn.conv2d(x, kernel, strides=[1,1,1,1], padding='SAME') #padding='SAME' 때문에

In [221]:
def pooling(x): # 2x2 를 1x1로 maxpooling 하는 함수
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [222]:
kernel_1 = weight_variable([5,5,1,32]) # 1개의 input에 대해서 5x5의 커널을 32개의 채널로 만듬
b_conv_1 = bias_variable([32]) #커널의 채널수에 맞는 bias 만듬

In [223]:
conv_1 = tf.nn.relu(convolution(x_image, kernel_1) + b_conv_1) #합성곱 후 활성함수에 넣은 activation map
pool_1 = pooling(conv_1) #activation map maxpooling으로 간소화 48x48 -> 24x24

In [224]:
kernel_2 = weight_variable([5,5,32,64]) # 32개의 input에 대해서 5x5의 커널을 64개의 채널로 만듬
b_conv_2 = bias_variable([64]) #커널의 채널수에 맞는 bias

In [225]:
conv_2 = tf.nn.relu(convolution(pool_1, kernel_2) + b_conv_2)
pool_2 = pooling(conv_2) # 위 행위 반복 24x24 -> 12x12

In [226]:
kernel_3 = weight_variable([5,5,64,128])
b_conv_3 = bias_variable([128])

In [227]:
conv_3 = tf.nn.relu(convolution(pool_2, kernel_3) + b_conv_3)
pool_3 = pooling(conv_3)# 12x12 -> 6x6

In [228]:
w_fc1 = weight_variable([6*6*128, 1024])
b_fc1 = bias_variable([1024])
pool_flat = tf.reshape(pool_3, [-1, 6*6*128])

In [229]:
h_fc1 = tf.nn.relu(tf.matmul(pool_flat, w_fc1) + b_fc1)
keep_prob = tf.placeholder("float")
h_fc1_dropout = tf.nn.dropout(h_fc1, keep_prob)

In [230]:
w_fc2 = weight_variable([1024,2])
b_fc2 = bias_variable([2])

In [231]:
y_conv = tf.nn.softmax(tf.matmul(h_fc1_dropout, w_fc2) + b_fc2) #softmax까지 써서 최종적인 판단 결과 도출

In [232]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(prediction, "float"))

In [236]:
sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer)
test_i, test_l = sess.run([image_stack, label_stack])
sess.run(train_step, feed_dict={x:test_i, y:test_l})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_26' with dtype float
	 [[{{node Placeholder_26}} = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_26', defined at:
  File "C:\Users\sfsfk\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sfsfk\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  F

In [144]:
for i in range(11):
    sess.run(iterator.initializer)
    batch = sess.run(next_element)
    train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y:batch[1], keep_prob:1.0})
    print(train_accuracy)
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1], keep_prob: 0.5})

FailedPreconditionError: Attempting to use uninitialized value Variable_35
	 [[{{node Variable_35/read}} = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_35)]]

Caused by op 'Variable_35/read', defined at:
  File "C:\Users\sfsfk\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sfsfk\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\Users\sfsfk\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\sfsfk\Anaconda3\lib\a